<a href="https://colab.research.google.com/github/yedomisol/checklist/blob/main/baseline_tag_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 태그 TF-IDF 베이스

**TF-IDF 기반 비교과 프로그램 추천 시스템**

1️⃣ 프로그램 태그 → TF-IDF 벡터화

2️⃣ 학생의 과거 수강 기록 → 가중 평균 벡터 생성

3️⃣ 학생 벡터 vs 프로그램 벡터 → 코사인 유사도 계산

4️⃣ 상위 후보 → 인기/품질로 리랭킹

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

In [2]:
# ------------------------------------------------------------
# 1. 텍스트 전처리
# ------------------------------------------------------------
def tags_to_text(tag):
    """태그 리스트/문자열을 TF-IDF 학습용 텍스트로 변환"""
    # 1) 리스트면 공백으로 합치고, 아니면 문자열로 안전 변환
    s = " ".join(map(str, tag)) if isinstance(tag, list) else ("" if pd.isna(tag) else str(tag))
    # 2) 콤마를 공백으로 바꿔 토큰 분리
    s = s.replace(",", " ")
    # 3) 소문자 + 중복 공백 제거
    s = " ".join(s.lower().split())
    return s

In [3]:
# ------------------------------------------------------------
# 2. TF-IDF 행렬 생성
# ------------------------------------------------------------
def build_tfidf_matrix(program_df, tag_col="tag",
                       min_df=3, ngram_range=(2, 4)):
    """프로그램 단위 TF-IDF 행렬 (안전한 폴백 포함)"""
    corpus = program_df[tag_col].apply(tags_to_text)
    # 1차 시도: 설정값 그대로
    try:
        vectorizer = TfidfVectorizer(analyzer="char", ngram_range=ngram_range, min_df=min_df)
        X = vectorizer.fit_transform(corpus)
        if X.shape[1] > 0:
            return vectorizer, X
    except ValueError:
        pass

    # 2차 시도: 용어가 매우 적은 데이터셋 폴백
    for params in (
        dict(analyzer="char", ngram_range=(2, 3), min_df=1),
        dict(analyzer="word", ngram_range=(1, 2), min_df=1),
    ):
        try:
            vectorizer = TfidfVectorizer(**params)
            X = vectorizer.fit_transform(corpus)
            if X.shape[1] > 0:
                return vectorizer, X
        except ValueError:
            continue

    # 마지막 폴백: 1열 영(0) 희소행렬
    from scipy.sparse import csr_matrix
    return TfidfVectorizer(), csr_matrix((len(corpus), 1))

In [4]:
# ------------------------------------------------------------
# 3. 학생 벡터 생성 (가중 평균)
# ------------------------------------------------------------
def build_student_vector(student_df, program_df, X,
                         program_id_col="program_id", ref_date=None):
    """학생의 과거 수강 TF-IDF 가중 평균 벡터(기준일 이전 기록만)"""
    if ref_date is None:
        ref_date = pd.Timestamp.today().normalize()

    # 기준일 이전 이력만 사용
    tmp = student_df.copy()
    tmp["start"] = pd.to_datetime(tmp.get("start"), errors="coerce")
    tmp["end"]   = pd.to_datetime(tmp.get("end"),   errors="coerce")
    end_eff = tmp["end"].fillna(tmp["start"])
    tmp = tmp[end_eff <= ref_date]
    if tmp.empty:
        return np.zeros((1, X.shape[1]))

    # 프로그램 매칭(정렬 맞추기 위해 program_df 서브셋 인덱스와 merge)
    sel = program_df[[program_id_col]].reset_index().rename(columns={"index":"prog_idx"})
    tmp = tmp.merge(sel, on=program_id_col, how="inner")
    if tmp.empty:
        return np.zeros((1, X.shape[1]))

    # 가중치 계산
    def recency_weight(dates, ref, half_life_days=180):
        dd = (ref - pd.to_datetime(dates, errors="coerce")).dt.days.fillna(365)
        return np.exp(-np.log(2) * (dd / half_life_days))

    w_finish = tmp["completed"].astype(str).str.contains("이수", na=False).astype(float)
    w_satis  = pd.to_numeric(tmp.get("satisfy", 0), errors="coerce").fillna(0)
    w_satis  = (w_satis - w_satis.min()) / (w_satis.max() - w_satis.min() + 1e-9)
    w_rec    = recency_weight(tmp.get("end", ref_date), ref=ref_date)
    w        = 0.6*w_finish + 0.25*w_satis + 0.15*w_rec

    # X에서 해당 프로그램 행만 뽑아 동일 순서로 가중 평균
    rows = tmp["prog_idx"].to_list()
    X_sel = X[rows].toarray()
    vec = np.average(X_sel, axis=0, weights=w.values)
    return vec.reshape(1, -1)

In [5]:
# ------------------------------------------------------------
# 4. 추천 (TF-IDF 유사도 + 인기/품질 리랭킹)
# ------------------------------------------------------------
def recommend_tfidf(df, student_id, top_n=10, candidate_k=100,
                    weights=(0.8, 0.1, 0.1), upcoming_days=90, ref_date=None):
    # 기준일
    ref_date = pd.Timestamp.today().normalize() if ref_date is None else pd.to_datetime(ref_date).normalize()

    # 프로그램 메타(중복 제거)
    base_cols = ["program_id","program_name","tag","start","end","type","method"]
    program_df = df.copy()
    if "start" in program_df.columns:
        program_df = program_df.sort_values("start")
    program_df = (program_df.drop_duplicates("program_id")
                               .loc[:, [c for c in base_cols if c in program_df.columns]]
                               .reset_index(drop=True))
    if "tag" not in program_df.columns:
        # 태그 컬럼이 없으면 빈 문자열로 대체하여 TF-IDF가 실패하지 않도록 처리
        program_df["tag"] = ""
    program_df["start"] = pd.to_datetime(program_df["start"], errors="coerce")
    program_df["end"]   = pd.to_datetime(program_df["end"],   errors="coerce")
    # 효과적 시작/종료 시점 (결측 안전 처리)
    program_df["start_eff"] = program_df["start"].fillna(program_df["end"])
    program_df["end_eff"]   = program_df["end"].fillna(program_df["start"])

    # 프로그램 단위 집계(만족/가입/이수) — 학생행에서 안전하게 집계
    agg = (df.groupby("program_id")
             .agg(count_enroll=("count_enroll","max"),   # 프로그램 단위라면 max/first
                  count_complete=("count_complete","max"),
                  satisfy_mean=("satisfy","mean"))
             .reset_index())
    program_df = program_df.merge(agg, on="program_id", how="left")

    # TF-IDF
    vectorizer, X = build_tfidf_matrix(program_df, tag_col="tag")

    # 학생 로그(기준일 이전만)
    stu_logs = df.loc[df["student_id"].eq(student_id),
                      ["student_id","program_id","start","end","completed","satisfy"]].copy()
    if stu_logs.empty:
        return pd.DataFrame(columns=["program_id","program_name","score"])

    stu_logs["start"] = pd.to_datetime(stu_logs["start"], errors="coerce")
    stu_logs["end"]   = pd.to_datetime(stu_logs["end"],   errors="coerce")
    end_eff = stu_logs["end"].fillna(stu_logs["start"])
    stu_logs = stu_logs[end_eff <= ref_date]

    # 학생 벡터
    stu_vec = build_student_vector(stu_logs, program_df, X, ref_date=ref_date)
    if np.allclose(stu_vec, 0):
        return pd.DataFrame(columns=["program_id","program_name","score"])

    # 유사도
    sim = cosine_similarity(normalize(stu_vec), normalize(X)).ravel()
    # NaN 유사도 방지 (0으로 대체)
    sim = np.nan_to_num(sim, nan=0.0, posinf=0.0, neginf=0.0)
    program_df["content_sim"] = sim

    # 후보군: 기간 + 이미 수강 제외
    horizon = ref_date + pd.Timedelta(days=upcoming_days)
    cand = program_df[(program_df["end_eff"] >= ref_date) & (program_df["start_eff"] <= horizon)].copy()

    taken = set(stu_logs["program_id"])
    cand = cand[~cand["program_id"].isin(taken)]
    if cand.empty:
        return pd.DataFrame(columns=["program_id","program_name","score"])

    # 1차 후보 Top-k
    cand = cand.sort_values("content_sim", ascending=False).head(candidate_k).copy()

    # 인기/품질 점수
    def norm01(s):
        s = pd.to_numeric(s, errors="coerce").fillna(0)
        r = s.max() - s.min()
        return (s - s.min()) / r if r > 0 else s * 0

    finish_rate = cand["count_complete"] / cand["count_enroll"].replace(0, np.nan)
    cand["finish_rate"] = finish_rate.fillna(0)
    cand["popularity"]  = norm01(cand["finish_rate"])
    cand["quality"]     = norm01(cand["satisfy_mean"])

    # 최종 점수
    w_content, w_pop, w_qlt = weights
    cand["score"] = (w_content*cand["content_sim"] +
                     w_pop*cand["popularity"] +
                     w_qlt*cand["quality"])

    keep = ["program_id","program_name","type","method","start","end",
            "content_sim","popularity","quality","score","tag"]
    keep = [c for c in keep if c in cand.columns]
    return cand.sort_values("score", ascending=False).head(top_n)[keep]

In [7]:
df = pd.read_csv('merged_update.csv')

In [11]:
# ------------------------------------------------------------
# 5. 실행 예시 : 2024년 1월 1일 기준으로 이후 90일 내 프로그램 추천
# ------------------------------------------------------------
rec = recommend_tfidf(df, student_id='069552-6703795', top_n=10, candidate_k=100,
                    weights=(0.8, 0.1, 0.1), upcoming_days=90, ref_date='2024-01-01')
rec

Empty DataFrame
Columns: [program_id, program_name, score]
Index: []